In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
from hmmlearn.hmm import GaussianHMM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [7]:
# Prepare multivariate data
data = pd.read_csv('..\data\BrentOilPrices.csv')

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Maintenant prêt\AppData\Local\Temp\ipykernel_60164\2819560649.py:2: SyntaxWarning: invalid escape sequence '\d'
  data = pd.read_csv('..\data\BrentOilPrices.csv')


In [8]:
data.set_index('Date', inplace=True)

In [9]:
data['Price_Lagged'] = data['Price'].shift(1)

In [10]:
data.isna().sum()

Price           0
Price_Lagged    1
dtype: int64

In [11]:
data.dropna(inplace=True)

In [12]:
# Fit VAR model
model = VAR(data[['Price', 'Price_Lagged']]) # Include 'Price_Lagged' in the model
results = model.fit(maxlags=5)  # Adjust maxlags as needed
print(results.summary())

# Forecast
forecast = results.forecast(data[['Price', 'Price_Lagged']].values, steps=10) # Ensure forecast uses both variables
print(forecast)

c:\Users\Maintenant prêt\Desktop\PROJECTS\change_point_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\Maintenant prêt\Desktop\PROJECTS\change_point_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


LinAlgError: Singular matrix

# VAR Model

In [13]:
# Fit VAR model
model = VAR(data[['Price']])
results = model.fit(maxlags=5)  # Adjust maxlags as needed
print(results.summary())

# Forecast
forecast = results.forecast(data.values, steps=10)
print(forecast)

c:\Users\Maintenant prêt\Desktop\PROJECTS\change_point_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:559: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  _index = to_datetime(index)
c:\Users\Maintenant prêt\Desktop\PROJECTS\change_point_analysis\venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ValueError: Only gave one variable to VAR

# Markov-Switching ARIMA

In [14]:
# Prepare data
prices = data['Price'].values.reshape(-1, 1)

# Fit HMM model
model = GaussianHMM(n_components=2, covariance_type="diag", n_iter=1000)
model.fit(prices)

# Predict states
hidden_states = model.predict(prices)
print(hidden_states)

[0 0 0 ... 1 1 1]


# LSTM Model

In [18]:
# Prepare data for LSTM
def create_dataset(data, look_back=1):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 30
X, y = create_dataset(prices, look_back)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Build LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=100, batch_size=32, verbose=2)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


281/281 - 4s - 16ms/step - loss: 2515.8611
Epoch 2/100
281/281 - 4s - 14ms/step - loss: 1487.9716
Epoch 3/100
281/281 - 2s - 6ms/step - loss: 1004.9388
Epoch 4/100
281/281 - 2s - 7ms/step - loss: 691.8967
Epoch 5/100
281/281 - 3s - 9ms/step - loss: 485.0224
Epoch 6/100
281/281 - 2s - 8ms/step - loss: 345.7172
Epoch 7/100
281/281 - 2s - 9ms/step - loss: 249.8515
Epoch 8/100
281/281 - 2s - 8ms/step - loss: 181.4494
Epoch 9/100
281/281 - 3s - 11ms/step - loss: 130.8536
Epoch 10/100
281/281 - 2s - 7ms/step - loss: 92.9088
Epoch 11/100
281/281 - 3s - 9ms/step - loss: 64.9568
Epoch 12/100
281/281 - 3s - 9ms/step - loss: 45.0454
Epoch 13/100
281/281 - 2s - 7ms/step - loss: 31.0039
Epoch 14/100
281/281 - 2s - 8ms/step - loss: 21.5134
Epoch 15/100
281/281 - 2s - 7ms/step - loss: 15.3667
Epoch 16/100
281/281 - 2s - 9ms/step - loss: 11.4741
Epoch 17/100
281/281 - 3s - 9ms/step - loss: 8.8331
Epoch 18/100
281/281 - 3s - 9ms/step - loss: 7.2174
Epoch 19/100
281/281 - 3s - 12ms/step - loss: 6.0363
E